In [1]:
import nhl

In [2]:
help(nhl.skatersummary)

Help on function skatersummary in module nhl:

skatersummary(season, gametype='regular')
    Creates URL and query for skatersummary report



In [8]:
import pandas as pd

//anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [27]:
skaters = [skater for skater in nhl.skatersummary("20152016")]

In [29]:
len(skaters)

898

In [33]:
df = pd.DataFrame(skaters)

In [39]:
df.columns

Index([u'assists', u'faceoffWinPctg', u'gameWinningGoals', u'gamesPlayed',
       u'goals', u'otGoals', u'penaltyMinutes', u'playerFirstName',
       u'playerId', u'playerLastName', u'playerName', u'playerPositionCode',
       u'playerTeamsPlayedFor', u'plusMinus', u'points', u'pointsPerGame',
       u'ppGoals', u'ppPoints', u'seasonId', u'shGoals', u'shPoints',
       u'shiftsPerGame', u'shootingPctg', u'shots', u'timeOnIcePerGame'],
      dtype='object')

In [42]:
df.head()

,assists,faceoffWinPctg,gameWinningGoals,gamesPlayed,goals,otGoals,penaltyMinutes,playerFirstName,playerId,playerLastName,...,pointsPerGame,ppGoals,ppPoints,seasonId,shGoals,shPoints,shiftsPerGame,shootingPctg,shots,timeOnIcePerGame
0,6,0.0000,0,38,1,0,49,Greg,8474688,Pateryn,...,0.1842,0,0,20152016,0,0,21.9210,0.0312,32,1004.9210
1,1,0.2500,0,3,0,0,0,Ryan,8477451,Hartman,...,0.3333,0,0,20152016,0,0,14.3333,0.0000,3,554.3333
2,1,0.0000,0,6,0,0,2,Andrew,8474638,Campbell,...,0.1666,0,0,20152016,0,0,20.1666,0.0000,1,695.0000
3,50,0.4938,5,77,13,1,55,Ryan,8470612,Getzlaf,...,0.8181,6,22,20152016,0,1,24.5324,0.0730,178,1170.3506
4,16,0.0000,0,72,3,0,18,Jay,8470151,Bouwmeester,...,0.2638,1,2,20152016,0,0,29.7361,0.0285,105,1386.8472


In [48]:
df['timeOnIcePerGame'].head()

0    1004.9210
1     554.3333
2     695.0000
3    1170.3506
4    1386.8472
Name: timeOnIcePerGame, dtype: float64

In [59]:
df['playerPositionCode'].head()

0    D
1    R
2    D
3    C
4    D
Name: playerPositionCode, dtype: object

In [32]:
import plotly.plotly as py
import plotly.graph_objs as go

In [84]:
hover_text = []
for index, row in df.iterrows():
    hover_text.append(('Name: {playerFirstName} {playerLastName}<br>'+
                      'Plays for {team}' +
                      'Position: {position}<br>'+
                      '{goals} goals, {assists}, {points} points<br>'+
                      'played in {games} games').format(playerFirstName=row['playerFirstName'],
                                            playerLastName=row['playerLastName'],
                                            position=row['playerPositionCode'],
                                            goals=row['goals'],
                                            assists=row['assists'],
                                            points=row['points'],
                                            games=row['gamesPlayed'],
                                            team=row['playerTeamsPlayedFor'][0]))

df['text'] = hover_text

In [85]:
df['text'].head()

0    Name: Greg Pateryn<br>Plays for MPosition: D<b...
1    Name: Ryan Hartman<br>Plays for CPosition: R<b...
2    Name: Andrew Campbell<br>Plays for TPosition: ...
3    Name: Ryan Getzlaf<br>Plays for APosition: C<b...
4    Name: Jay Bouwmeester<br>Plays for SPosition: ...
Name: text, dtype: object

In [86]:
def makeTrace(df,name):
  return go.Scatter(
    y=[ points/shifts for points, shifts in zip(df['points'], df['shiftsPerGame'])],
    x=df['shiftsPerGame'],
    mode='markers',
    name=name,
	text=df['text']
)

In [90]:
layout = go.Layout(
    title='NHL Skaters productivity',
    xaxis=dict(
        title='Shifts per game',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=0,
    ),
    yaxis=dict(
        title='Productivity (points per shifts per game)',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=0,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
  	hovermode = 'closest'
)


In [91]:
positions = ['Defenceman', 'Right Wing', 'Center', 'Left Wing']
data = [makeTrace(df[(df.playerPositionCode == code[0])], code) for code in positions]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='nhl-stats')